# Running Native Functions


Two of the previous notebooks showed how to [execute semantic functions inline](./03-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

- Writing logic around how to run a prompt that changes the prompt's outcome.
- Using external data sources to gather data to concatenate into your prompt.
- Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the SK Python framework.


Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


In [ ]:
!python -m pip install semantic-kernel==0.5.1.dev0

In [1]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.OpenAI

In [9]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable

kernel = sk.Kernel()

if selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat"  # used later in the notebook
    azure_chat_service = AzureChatCompletion(
        service_id=service_id, deployment_name="gpt-35-turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_service(azure_chat_service)

# Configure OpenAI service
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "oai_chat"  # used later in the notebook
    oai_chat_service = OpenAIChatCompletion(
        service_id=service_id, ai_model_id="gpt-4-turbo-1106", api_key=api_key, org_id=org_id
    )
    kernel.add_service(oai_chat_service)

Let's create a **native** function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.


First, let's create our native function.


In [10]:
import random
from semantic_kernel.functions import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between 3-x.
    """

    @kernel_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$input}}` and generates that number of paragraphs about two Corgis on an adventure. `$input` is a default variable semantic functions can use.


In [11]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long. It must be this length.
"""

if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="story",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.create_function_from_prompt(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

generate_number_plugin = kernel.import_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")

In [12]:
# Run the number generator
generate_number_three_or_higher = generate_number_plugin["GenerateNumberThreeOrHigher"]
number_result = await generate_number_three_or_higher(kernel, input=6)
print(number_result)

6


In [15]:
story = await corgi_story.invoke(kernel, input=number_result.value)

_Note: depending on which model you're using, it may not respond with the proper number of paragraphs._

In [16]:
print(f"Generating a corgi story exactly {number_result.value} paragraphs long.")
print("=====================================================")
print(story)

Generating a corgi story exactly 6 paragraphs long.
Once upon a time in a quaint little town, there lived two adventurous Corgis named Max and Ruby. Max and Ruby loved exploring the world around them, and one sunny day, they decided to embark on a grand adventure to the nearby forest. As they trotted through the tall trees and lush greenery, their tails wagged with excitement.

As they journeyed deeper into the forest, they encountered all sorts of new and interesting sights and smells. They met friendly squirrels, playful rabbits, and even a wise old owl perched high in the trees. Max and Ruby were overjoyed by the beauty of the natural world and the kindness of the creatures they met along the way.

As the sun began to set, Max and Ruby realized they had ventured quite far from home. They looked at each other with concern, but then remembered the valuable lesson their parents had taught them - always stick together and help each other out. With newfound determination, they set out on

## Kernel Functions with Annotated Parameters

That works! But let's expand on our example to make it more generic.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of the Python's `Annotated` class to hold these variables.


In [17]:
import sys
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

if sys.version_info >= (3, 9):
    from typing import Annotated
else:
    from typing_extensions import Annotated

kernel = sk.Kernel()

if selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat"  # used later in the notebook
    azure_chat_service = AzureChatCompletion(
        service_id=service_id, deployment_name="gpt-35-turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_service(azure_chat_service)

# Configure OpenAI service
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "oai_chat"  # used later in the notebook
    oai_chat_service = OpenAIChatCompletion(
        service_id=service_id, ai_model_id="gpt-4-turbo-1106", api_key=api_key, org_id=org_id
    )
    kernel.add_service(oai_chat_service)

Let's start with the native function. Notice that we're add the `@kernel_function` decorator that holds the name of the function as well as an optional description. The input parameters are configured as part of the function's signature, and we use the `Annotated` type to specify the required input arguments.


In [19]:
import random
from semantic_kernel.functions import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between a min and a max.
    """

    @kernel_function(
        name="GenerateNumber",
        description="Generate a random number between min and max",
    )
    def generate_number(
        self,
        min: Annotated[int, "the minimum number of paragraphs"],
        max: Annotated[int, "the maximum number of paragraphs"] = 10,
    ) -> Annotated[int, "the output is a number"]:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(min, max))
        except ValueError as e:
            print(f"Invalid input {min} and {max}")
            raise e

In [20]:
generate_number_plugin = kernel.import_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")
generate_number = generate_number_plugin["GenerateNumber"]

plugin_name='GenerateNumberPlugin' description='Generate a random number between min and max' name='GenerateNumber' is_prompt=False stream_function=None parameters=[KernelParameterMetadata(name='min', description='the minimum number of paragraphs', default_value=None, type_='int', required=True, expose=True), KernelParameterMetadata(name='max', description='the maximum number of paragraphs', default_value=10, type_='int', required=True, expose=True)] return_parameter=KernelParameterMetadata(name='return', description='the output is a number', default_value=None, type_='int', required=True, expose=False) function=<bound method GenerateNumberPlugin.generate_number of <__main__.GenerateNumberPlugin object at 0x7fc317bfbe50>> plugins=KernelPluginCollection(plugins={'GenerateNumberPlugin': KernelPlugin(name='GenerateNumberPlugin', description=None, functions={'GenerateNumber': KernelFunction(plugin_name='GenerateNumberPlugin', description='Generate a random number between min and max', name

Now let's also allow the semantic function to take in additional arguments. In this case, we're going to allow the our CorgiStory function to be written in a specified language. We'll need to provide a `paragraph_count` and a `language`.


In [22]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
"""

if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.create_function_from_prompt(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

Let's generate a paragraph count.


In [23]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value
print(f"Generating a corgi story {num_paragraphs} paragraphs long.")

Generating a corgi story 3 paragraphs long.


We can now invoke our corgi_story function using the `kernel` and the keyword arguments `paragraph_count` and `language`.

In [24]:
# Pass the output to the semantic story function
desired_language = "Spanish"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

In [25]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 3 paragraphs long in Spanish.
Había una vez dos corgis llamados Coco y Tito. Eran los mejores amigos y les encantaba ir en aventuras juntos. Un día, decidieron explorar el bosque cercano a su casa. Se adentraron en el bosque y descubrieron un hermoso arroyo donde jugaron y se divirtieron durante horas.

Mientras exploraban, Coco y Tito se encontraron con un pajarito herido. Sin dudarlo, decidieron llevarlo a casa y cuidarlo hasta que estuviera completamente recuperado. A medida que pasaban los días, el pajarito se recuperó y se convirtió en un amigo más para Coco y Tito. Juntos, disfrutaron de muchas aventuras y aprendieron que la amistad y la compasión son los valores más importantes en la vida.

Coco y Tito demostraron que la verdadera amistad y el amor por los demás pueden hacer del mundo un lugar mejor. Aprendieron que siempre hay espacio para ayudar a los demás y que las aventuras son mucho más emocionantes cuando se comparten con amigos. Y así, Coco, Tito

## Calling Native Functions within a Semantic Function

One neat thing about the Semantic Kernel is that you can also call native functions from within Prompt Functions!

We will make our CorgiStory semantic function call a native function `GenerateNames` which will return names for our Corgi characters.

We do this using the syntax `{{plugin_name.function_name}}`. You can read more about our prompte templating syntax [here](../../../docs/PROMPT_TEMPLATE_LANGUAGE.md).


In [26]:
import random
from semantic_kernel.functions import kernel_function


class GenerateNamesPlugin:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @kernel_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @kernel_function(description="Generate character names", name="generate_names")
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {"Hoagie", "Hamilton", "Bacon", "Pizza", "Boots", "Shorts", "Tuna"}
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

In [27]:
generate_names_plugin = kernel.import_plugin(GenerateNamesPlugin(), plugin_name="GenerateNames")
generate_names = generate_names_plugin["generate_names"]

In [33]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
- The two names of the corgis are {{GenerateNames.generate_names}}
"""

In [34]:
if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="corgi-new",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.create_function_from_prompt(
    function_name="CorgiStoryUpdated",
    plugin_name="CorgiPluginUpdated",
    prompt_template_config=prompt_template_config,
)

In [35]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value

In [36]:
desired_language = "French"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

In [37]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 5 paragraphs long in French.
Il était une fois, dans une petite ville pittoresque, deux adorables corgis nommés Bacon et Shorts. Ces deux petits chiens étaient inséparables et aimaient partir à l'aventure ensemble.

Un jour, Bacon et Shorts décidèrent de partir explorer la forêt voisine. Ils couraient joyeusement à travers les sentiers, s'arrêtant de temps en temps pour renifler les fleurs sauvages et regarder les oiseaux dans les arbres.

Alors qu'ils marchaient, ils rencontrèrent un lapin effrayé qui avait perdu son chemin. Sans hésiter, Bacon et Shorts s'approchèrent du lapin et lui offrirent leur aide. Ensemble, ils guidèrent le lapin de retour à son terrier, où sa famille l'attendait avec impatience.

Le lapin était tellement reconnaissant qu'il invita Bacon et Shorts à un festin dans la clairière. Les corgis furent enchantés par la gentillesse du lapin et par la délicieuse nourriture qu'il leur offrit. Ils réalisèrent que même les plus petites actions de 

### Recap

A quick review of what we've learned here:

- We've learned how to create native and prompt functions and register them to the kernel
- We've seen how we can use Kernel Arguments to pass in more custom variables into our prompt
- We've seen how we can call native functions within a prompt.
